#@ComponentScan
Class 위에 @Component 어노테이션을 주입하면 그 어노테이션만 찾아서 스프링빈에 연결해주는데 하위 상속받은 항목이있을땐 @AutoWired를 생성자위에 적어서 상속되는 항목들도 다 찾아서 스캔해줌 
#탐색위치와 기본 스캔 대상
@componetscan {
basePackages = "hello.core.member",
} member패키지 안에서 탐색위치를 지정함
디폴트면 hello.core아래부터 다 뒤짐(ComponetScan)이 붙은 위치를뒤짐
{권장하는 방법
개인적으로 즐겨 사용하는 방법은 패키지 위치를 지정하지 않고, 설정 정보 클래스의 위치를 프로젝트
최상단에 두는 것이다. 최근 스프링 부트도 이 방법을 기본으로 제공한다.
예를 들어서 프로젝트가 다음과 같이 구조가 되어 있으면
com.hello
com.hello.serivce
com.hello.repository
com.hello 프로젝트 시작 루트, 여기에 AppConfig 같은 메인 설정 정보를 두고, 
@ComponentScan 애노테이션을 붙이고, basePackages 지정은 생략한다.
이렇게 하면 com.hello 를 포함한 하위는 모두 자동으로 컴포넌트 스캔의 대상이 된다. 그리고 프로젝트
메인 설정 정보는 프로젝트를 대표하는 정보이기 때문에 프로젝트 시작 루트 위치에 두는 것이 좋다}->국룰임

#컴포넌트 스캔 기본대상
@Component 뿐만아니라 @Controller, @Service, @Repository,@Configuration이 자동으로 대상에 포함됨 ->스프링이 지원하는기능임(자바언어가 지원하는건 X)

@Controller(스프링 MVC 컨트롤러로 인식), @Service(별거안함), @Repository(스프링 데이터 접근계층으로 인식하고 데이터 계층의 예외를 스프링 예외로 변환해줌),@Configuration(설정정보로 인식하고 싱글톤을 유지하도록 추가처리함)들이있으면 부가기능도 수행

#필터
excludeFilters는 간혹 쓰이지만 includefilters는 거의 안씀
#중복 등록과 충돌
자동빈등록 vs 자동빈등록
-> 오류발생
수동빈등록 vs 자동빈등록
-> 수동빈등록이 우선권을 가지고 자동빈을 오버라이딩함
(최근에는 수동 빈 등록과 자동 빈 등록이 충돌나면 오류가 발생하도록 기본 값을 바꿈(잘못하면 코드가 꼬여버림))

##의존관계 자동 주입
#1 생성자 주입
#2 수정자 주입(Setter주입)
#3 필드 주입
#4 일반매서드 주입
생성자 주입
- 생성자 호출시점에 딱 1번만 호출되는것을보장
- "불변, 필수" 의존관계에 사용
수정자 주입
@Component
public class OrderServiceImpl implements OrderService{

    private  MemberRepository memberRepository;
    private  DiscountPolicy discountPolicy;
    
    @Autowired
    public void setDiscountPolicy(DiscountPolicy discountPolicy) {
        this.discountPolicy = discountPolicy;
    }
    @Autowired
    public void setMemberRepository(MemberRepository memberRepository) {
        this.memberRepository = memberRepository;
    }
이런식으로 주입 set~~
-"선택,변경" 가능성있는 의존관계에 사용
private int age;
public void setAge(int age){
    this.age = age;
}
public void getAge(){
    return = age;
}
이게 국룰임(set, get)

필드주입
필드에 그냥 바로 주입하는것 
외부에서 변경불가해서 테스트하기가어려워서 안티코드 안쓰면된다
일반매서드주입
이것도 안하는게 나음


#

#옵션처리
주입할 스프링빈이없어도 동작해야할때도있는데
@AutoWired(required=False) 이런거하면 댐
@Nullable, Optional<Member> 이런것도 좋음

#생성자 주입을 선택하라
과거에는 수정자주입, 필드주입등을 썼지만 왠만하면 생성자 주입을 사용하는게 좋음.
이유-> "불변"(수정자주입시 setXxx메서드를 public으로 열어놔야함) ,"누락" final키워드 설정가능함(생성자에서 혹시라도 값이설정되지않는 오류를 컴파일 시점에서 막아줌) 나머지는 생성자 이후로 호출되니까 final 키워도 사용 불가

#롬복과 최신 트랜드
막상 개발해보면 왠만하면 다 불변 근데 귀찮다 편리하게 사용하는방법이있음
getter setter 자동으로 만들어줌 final이붙은거도 @Requried어쩌구적으면 자동으로 만들어줌 그 this 이지랄 안해도댐

#조회 빈이 2개이상
@Autowired 필드명 매칭
@Quilifier -> @Quilifier끼리매칭
@Primary 사용 편해서 자주쓰긴함

annotation에는 상속이라는 개념이없음 여러 어노테이션을 모아서 사용하는 기능은 스프링이 지원해주는 기능이다.


#조회한 빈이 모두 필요할때 (List, Map)
동적으로 받아야할때 Map을이용하면 다형성을 유지하면 사용하기 쉽다

#자동, 수동의 올바른 실무 운영 기준

자동을 기본으로 사용함

#수동을 사용할때는? 
어플리케이션은 크게 업무로직과 지원로직으로 나뉨
업무로직 빈-> 웹을 지원하는 컨트롤러, 핵심 비즈니스 로직이있는 서비스, 데이터 계층의 로직을 처리하는 리포지토리등,
기술지원 빈 -> 기술적인 문제 혹은 AOP(공통관심사)를 처리할때 주로 사용된다 공통로그 처리, 데이터베이스연결과같은 빈

업무로직빈 => 자동기능을 사용하는게 좋다
기술지원 빈=> 수동 빈등록을 사용해서 명확하게 들어내는것이 좋음 자동으로 할거면 특정 패키지에 같이 묶어두는게 좋음

##콜백이란?
스프링빈이 생성되거나 소멸되기전에 메서드를 호출하는 기능

데이터베이스 커넥션 :  풀 서버와 데이터베이스와 연결을 미리 맺어놓고 고객 요청이올때 연결온걸 재활용하는거
네트워크 소켓 : 미리 연결을 해두고(소켓을열어두고) 안전하게 종료하는 작업 -> 객체의 초기화, 종료 작업이 필요하다

스프링 빈은 객체를 생성하고, 의존관계 주입이 다 끝난 다음에야 필요한 데이터를 사용할 수 있는 준비가 완료된다. 따라서 초기화 작업은 의존관계 주입이 모두 완료되고 난 다음에 호출해야 한다. 그런데 개발자가 의존관계 주입이 모두 완료된 시점을 어떻게 알 수 있을까?
스프링은 의존관계 주입이 완료되면 스프링 빈에게 콜백 메서드를 통해서 초기화 시점을 알려주는 다양한 기능을 제공한다. 또한 스프링은 스프링 컨테이너가 종료되기 직전에 소멸 콜백을 준다. 따라서 안전하게 종료 작업을 진행할 수 있다.

#스프링빈의 이벤트 라이프사이클
스프링 컨테이너 생성 -> 스프링 빈 생성-> 의존관계 주입 -> 초기화 콜백 -> 사용 -> 소멸전 콜백 -> 스프링 종료
-초기화 콜백 -> 빈이 생성되고, 빈의 의존관계 주입이 완료된 후 호출
-소멸전 콜백 -> 빈이 소멸되기 직전에 호출

#객체의 생성과 초기화를 분리하자(단일체계원칙)

#스프링은 크게 3가지 방법으로 bean 생명주기콜백을 지원한다

1 인터페이스 IntiallizingBean(초기화빈) ,DisposableBean(종료빈)
단점 스프링 전용 인터페이스. 해당 코드가 스프링 전용 인터페이스에 의존 초기화, 소멸메서드의 이름을 변경 못함, 내가 코드를 고칠수 없는 외부 라이브러리에 적용할수없음
!!거의 안씀!!

2 설정정보에 초기화 메서드, 종료 메서드 지정(@Bean(initMethod = "init", destroyMethod = "close"))
매서드이름을 자유롭게 줄수있음
스프링빈이 스프링 코드에 의존하지않는다
!!코드가 아니라 설정정보를 사용하기때문에 코드를 고칠수없는 외부라이브러리에도 초기화, 종료 메서드를 적용가능!!
destroyMethod 에는 기능이 하나있는데 close나 shutdown 이라는 메서드를 자동으로 호출해줘서 뭐 안적어줘도 호출해줌 공백으로하면 추론기능 동작 X

#3 @PostConstruct @PreDestory 어노테이션 지원 이거쓰면됨
javax에서 지원하는거라 스프링말고도 쓸수잇음 권장방법 편리함
컴포넌트 스캔과 잘 어울림 유일한 단점은 외부라이브러리는 적용 못한다 외부라이브러리쓸꺼면 Bean설정으로 쓰자


##빈 스코프
스코프는 빈이 존재할수있는 범위라는것임
스프링은 다양한 스코프를 지원
"싱글톤" : 기본 스코프, 스프링 컨테이너의 시작과종료까지 유지되는 가장 넓은범위
"프로토타입" : 스프링컨테이너는 프로토타입 빈의 생성과 의존관계까지만 관여하는 매우짧은 범위 스코프
"웹 관련 스코프"
1 request 웹 요청이 들어오고 나갈때까지 유지되는 스코프
2 session 웹 세션이 생성되고 종료될때까지 유지되는 스코프(로그인에서씀)
3 application 웹의 서블릿 컨텍스와 같은 범위로 유지되는 스코프
싱글톤 프로토타입 request정도가 중요함
@Scope("prototype") 이런식으로 씀

#프로토타입 스코프
싱글톤 스코프의 빈을 조회하면 컨테이너는 항상 같은 인스턴스의 스프링빈을 반환하지만 프로토타입 스코프를 스프링 컨테이너에 조회하면 항상 새로운 인스턴스를 생성해서 반환함

#순서
1. 프로토타입 스코프의 빈을 스프링 컨테이너에 요청한다.
2. 스프링 컨테이너는 이 시점에 프로토타입 빈을 생성하고, 필요한 의존관계를 주입한다.
3. 스프링 컨테이너는 생성한 프로토타입 빈을 클라이언트에 반환한다.
4. 이후에 스프링 컨테이너에 같은 요청이 오면 항상 새로운 프로토타입 빈을 생성해서 반환한다

#핵심은 스프링컨테이너는 프로토타입 빈을 생성, DI주입, 초기화까지만 처리한다는것 프로토타입 관리할책임은 프로토타입 빈을 받은 클라이언트에게 있음
종료메서드가 호출되지 않는다.

#프로토타입 스코프를 싱글톤 빈과 함께 사용시 문제점
같은걸 계속쓸수가있음 처음 주입받을때 들어오면 같은게 계속 사용됨
원하는건 사용할때마다 새로 생성해서 쓰는걸 원하는것
즉 싱글톤빈과 주입받은 의존관계가 계속 유지되는게 문제임
*참고 빈 여러개에서 주입받으면 새롭게 생성되서 괜찮긴함 하지만 사용할떄마다 새로 생성되는것은 역시 아님
#해결방법 Provider로 문제해결
싱글톤 빈과 프로토타입빈을 함께 사용할때 어떻게하면 새로 프로토타입빈이 생성되게 만들수있을까?

참고로 DL이라는건 직접 필요한 의존관계를 찾는것을말함 Lookup
 static class ClientBean {

        @Autowired
        private ObjectProvider<PrototypeBean> prototypeBeanProvider;

        public int logic() {
            PrototypeBean prototypeBean = prototypeBeanProvider.getObject();
            prototypeBean.addCount();
            int count = prototypeBean.getCount();
            return count;

이런식으로 하면 프로토타입빈이 불러올때마다 생김
대신 찾아주는 느낌 (DL) 근데 그게 프로토타입 빈인거임
기능 단순하고 별도라이브러리 필요없고 스프링에 의존함

#JSR-330 Provider
이건 자바표준 javax~
근데 라이브러리 추가해줘야함 gradle에 

!정리
프로토타입빈을 직접적으로 쓸일은 거의없긴하다 근데 중요한 개념이라 배워놓긴해야함

##웹 스코프
특징 - 웹 환경에서만 동작함, 해당 스코프의 종료시점까지 관리함(프로토타입과 다르게) 따라서 종료 메서드가 호출된다
종류 "request" http요청 하나가 들어오고 나갈때 까지 유지되는 스코프 각각의 http요청마다 별도의 빈 인스턴스가 생성되고 관리됨
gradle에 웹 임포트하면 웹 8080포트에서 켜짐

*동시에 여러 HTTP요청이오면 정확이 어떤 요청이 남긴 로그인지 구분하기 어려움. 이럴때 사용하기 알맞은게 request스코프
공통 포맷 -> [UUID][requestURL][message] -> uuid가 유니크아이디임 

#스코프와 프록시 @Scope에서 proxyMode= ScopeProxymode.TARGET_CLASS 로 파라미터 넣어놓고 그냥 하면 댐
이렇게하면 MyLogger의 가짜 프록시 클래스를 만들어두고 주입해줌